In [1]:
import json
import pandas as pd
import openpyxl 
import http.client
import time 
import socket

### Récupérer tous les catalogues de jeux de données depuis l'API BdF

In [4]:
''' API BDF ''' 
# https://developer.webstat.banque-france.fr/node/295

conn = http.client.HTTPSConnection("api.webstat.banque-france.fr")

headers = {
    'X-IBM-Client-Id': "4a7786bb16188911b4cce4006fed2088",
    'accept': "application/json"
    }

conn.request("GET", "/webstat-fr/v1/catalogue?format=json", headers=headers)

res = conn.getresponse()
data = res.read()

text = data.decode("utf-8")
json_object = json.loads(text)
df = pd.DataFrame(json_object)
df.to_excel('data/result/bdf/bdf_datasets.xlsx', index = False)

In [5]:
lst_data_1 = df['name'].iloc[:10].tolist()
lst_data_2 = df['name'].iloc[10:20].tolist()
lst_data_3 = df['name'].iloc[20:30].tolist()
lst_data_4 = df['name'].iloc[30:40].tolist()
lst_data_5 = df['name'].iloc[40:].tolist()



In [10]:
for i in range(1, 6):
    lst_name = f"lst_data_{i}"
    print(f"{lst_name}: {globals()[lst_name]}")



lst_data_1: ['DIREN', 'MIR1', 'DWA1', 'CONJ2', 'MNA', 'MIR', 'IVF1', 'AME', 'FCI', 'CFANF']
lst_data_2: ['BPM6', 'EXR', 'ACPR', 'PEA', 'FATSO', 'CEFIT', 'BLS', 'OFC', 'QFAGG', 'TCN2']
lst_data_3: ['DSE', 'ENDET', 'PME', 'ICB', 'RPP', 'GST', 'CONS', 'SC1', 'STS', 'CPP']
lst_data_4: ['CFT', 'FM', 'SA', 'CONJ', 'ESTR', 'ECOFI', 'CDIS', 'CFTDC', 'CNF', 'ICP']
lst_data_5: ['SEC', 'SI_AI', 'ESA', 'FTID', 'SAFE', 'BSI', 'RA_T', 'DET', 'BSI1']


In [52]:
lst_data_1.remove('CONJ2')
lst_data_1.remove('FM')

### Récupérer le contenu (indicateurs disponibles) des jeux de données 

changer lst_data_1 pour lst_data_2, lst_data_3... 

In [82]:
''' API BDF ''' 
#https://developer.webstat.banque-france.fr/node/295


conn = http.client.HTTPSConnection("api.webstat.banque-france.fr")

headers = {
    'X-IBM-Client-Id': "b983b4cb-7e87-4c66-914c-c254ff3d9cfb",
    'accept': "application/json"
    }

df2 = pd.DataFrame()
dataset_ko = []
#for i in range(1,6):
#    for name in globals()[f"lst_data_{i}"]:
for name in lst_data_1:
        liste_df = []
        try:
            conn.request("GET", f'/webstat-fr/v1/data/{name}?format=json', headers=headers)
            #conn.sock.settimeout(10)
            res = conn.getresponse()
            data = res.read()
            sample = data.decode('utf-8')
            json_ob = json.loads(sample)
        except (http.client.HTTPException, json.JSONDecodeError, socket.timeout) as e:
            print(f"Erreur lors de la requête : {e}")
            dataset_ko.append(name)
            continue
            

        time.sleep(5)
        datasetName = json_ob["datasetName"]  # Nom du jeu de données
        liste_cles = []
        

        for serie in json_ob["seriesObs"]:
            title = serie["ObservationsSerie"]["title"]
            frequency = serie["ObservationsSerie"]["frequency"]
            observations = serie["ObservationsSerie"]["observations"]
        
            
            for observation in observations:
                liste_cles = list(observation['ObservationPeriod'].keys())
            
            df2 = df2.append({
            "datasetName": datasetName,
            "title": title,
            "frequency": frequency,
            "filtres": liste_cles
            }, ignore_index=True)

   
    


df2.to_excel(f"data_result/bdf/bdf_details_4.xlsx", index=False)  # Enregistrer le DataFrame au format Excel

# Confirmer que le fichier a été créé
print("Le fichier a été généré avec succès.")

KeyError: 'datasetName'

### Fonction pour récupérer le contenu d'un dataset

In [11]:
def getContent(df_name, title, path):
    conn = http.client.HTTPSConnection("api.webstat.banque-france.fr")

    headers = {
        'X-IBM-Client-Id': "4a7786bb16188911b4cce4006fed2088",
        'accept': "application/json"
        }

 
    workbook = openpyxl.Workbook()

    conn.request("GET", f"/webstat-fr/v1/data/{df_name}?format=json", headers=headers)
    res = conn.getresponse()
    data = res.read()
    sample = data.decode('utf-8')
    json_ob = json.loads(sample)

    datasetName = json_ob["datasetName"]  # Nom du jeu de données

    data = []
    for serie in json_ob["seriesObs"]:
        obs_title = serie["ObservationsSerie"]["title"]
        if obs_title == 'Belgique, Indice de ventes au détail (CVS-CJO)':
            observations = serie["ObservationsSerie"]["observations"]
            
            for observation in observations:
                period_id = observation["ObservationPeriod"]["periodId"]
                period_first_date = observation["ObservationPeriod"]["periodFirstDate"]
                period_name = observation["ObservationPeriod"]["periodName"]
                value = observation["ObservationPeriod"]["value"]
                
                data.append({
                    "datasetName": datasetName,
                    "title": obs_title,
                    "periodId": period_id,
                    "periodFirstDate": period_first_date,
                    "periodName": period_name,
                    "value": value
                })

    df = pd.DataFrame(data)
    df.to_excel(path+title+".xlsx",index=False)
    return df 

In [12]:
df = getContent('STS',"Belgique, Indice de ventes au détail (CVS-CJO)","data/result/bdf/" )